In [ ]:
from summer import CompartmentalModel
import matplotlib.pyplot as plt

# Chapter 3

In [ ]:
# Key input parameters in model
t_step = 1  # Size of time step (days)
total_population = 100000  # Total population size
ave_preinfous = 2  # Average pre-infectious period (days)
ave_infous = 2  # Average duration of infectiousness (days)
r0 = 2  # R0

# Parameters derived from the above, no need to consider time step, which is done automatically
beta = r0 / ave_infous  # No need to divide by population because this is part of infection_frequency
infous_rate = 1 / ave_preinfous
rec_rate = 1 / ave_infous

infectious_seed = 1

In [ ]:
# Set up the SEIR model
compartments = ("Susceptible", "Pre-infectious", "Infectious", "Immune")
seir_model = CompartmentalModel(
    times=[0, 200],
    compartments=compartments,
    infectious_compartments=["Infectious"],
    timestep=t_step,
)
seir_model.set_initial_population(
    distribution={
        "Susceptible": total_population - infectious_seed, 
        "Infectious": infectious_seed
    }
)
seir_model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=beta,
    source="Susceptible",
    dest="Pre-infectious"
)
seir_model.add_transition_flow(
    name="progression", 
    fractional_rate=infous_rate,
    source="Pre-infectious", 
    dest="Infectious"
)
seir_model.add_transition_flow(
    name="recovery", 
    fractional_rate=rec_rate, 
    source="Infectious", 
    dest="Immune"
)
seir_model.request_output_for_flow(
    name="incidence", 
    flow_name="progression"
)
seir_model.request_output_for_flow(
    name="infection", 
    flow_name="infection"
)

In [ ]:
seir_model.run()

In [ ]:
# No more analysis needed, straight to plotting
fig, ax = plt.subplots(1, 1, dpi=120)
ax.plot(seir_model.times, seir_model.outputs.T[2], color="red")
ax.set_xlim(0, 1e2)
ax.set_ylim((0, 2e4))
ax.set_xlabel("Time (days)")
ax.set_ylabel("Number of infectious persons")

In [ ]:
# Set up the SEIR model
compartments = ("Susceptible", "Pre-infectious", "Infectious", "Immune")
seir_model = CompartmentalModel(
    times=[0, 200],
    compartments=compartments,
    infectious_compartments=["Infectious"],
    timestep=t_step,
)
seir_model.set_initial_population(
    distribution={
        "Susceptible": total_population - infectious_seed, 
        "Infectious": infectious_seed
    }
)
seir_model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=beta,
    source="Susceptible",
    dest="Pre-infectious"
)
seir_model.add_transition_flow(
    name="progression", 
    fractional_rate=infous_rate,
    source="Pre-infectious", 
    dest="Infectious"
)
seir_model.add_transition_flow(
    name="recovery", 
    fractional_rate=rec_rate, 
    source="Infectious", 
    dest="Immune"
)
seir_model.request_output_for_flow(
    name="incidence", 
    flow_name="progression"
)
seir_model.request_output_for_flow(
    name="infection", 
    flow_name="infection"
)

## Model 3.2

In [ ]:
# Recycle the parameters that are unchanged, but change the following:
ave_preinfous = 8
ave_infous = 7
r0 = 13
life_expectancy = 70

# Derived parameters
beta = r0 / ave_infous
infous_rate = 1 / ave_preinfous
rec_rate = 1 / ave_infous
m_rate = 1 / life_expectancy / 365

# Create the SEIR model with demographic processes
compartments = ("Susceptible", "Pre-infectious", "Infectious", "Immune")
seir_demog_model = CompartmentalModel(
    times=[0, 36500],
    compartments=compartments,
    infectious_compartments=["Infectious"],
    timestep=t_step,
)
seir_demog_model.set_initial_population(
    distribution={
        "Susceptible": total_population - infectious_seed, 
        "Infectious": infectious_seed
    }
)
seir_demog_model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=beta,
    source="Susceptible", 
    dest="Pre-infectious"
)
seir_demog_model.add_transition_flow(
    name="progression", 
    fractional_rate=infous_rate,
    source="Pre-infectious", 
    dest="Infectious"
)
seir_demog_model.add_transition_flow(
    name="recovery", 
    fractional_rate=rec_rate, 
    source="Infectious", 
    dest="Immune"
)
seir_demog_model.add_universal_death_flows(
    "universal_death",
    death_rate=m_rate
)
seir_demog_model.add_replacement_birth_flow(
    "births",
    "Susceptible"
)

seir_demog_model.request_output_for_flow(
    name="incidence", 
    flow_name="progression"
)

In [ ]:
seir_demog_model.run(solver="euler", step_size=t_step)

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=120)
ax.plot(seir_demog_model.times, seir_demog_model.outputs.T[0], color="black")
ax.plot(seir_demog_model.times, seir_demog_model.outputs.T[-1], color="green")
ax.set_ylim((0, 1e5))
ax.set_ylabel("Number of susceptible or immune persons")
ax.set_xlabel("Time (days)")

right_ax = ax.twinx()
right_ax.plot()
right_ax.plot(seir_demog_model.times, seir_demog_model.derived_outputs["incidence"], color="purple")
right_ax.set_ylim((0, 300))
right_ax.set_ylabel("Number of new infectious persons per time step")